In [7]:
import json
import numpy as np

In [3]:
def load_json(filename):
    with open(f"{filename}", 'r') as fp:
        data = json.load(fp)
    return data

In [9]:
resources = '/home/nikolaisv/study/intership/Analysis-of-Socio-Technical-Congruence/resources'
FDM = np.array(load_json(f"{resources}/fileDependencyMatrix"))
UFI = load_json(f"{resources}/userFilesIds")

In [12]:
UFI['1']

[25, 26]

In [14]:
FDM.shape

(7727, 7727)

In [22]:
shape = len(UFI.keys()), FDM.shape[1]
shape

(1681, 7727)

In [23]:
UFI_arr = np.zeros(shape)
for key, value in UFI.items():
    for v in value:
        UFI_arr[int(key)] = v

In [25]:
FDM_filtered = FDM
CN = UFI_arr @ FDM_filtered @ UFI_arr.T

In [26]:
CN.shape

(1681, 1681)

In [33]:
def print_(M):
    print(f"{M.min()}, {M.mean()}, {M.max()}")

In [34]:
CN_n = CN / CN.max()
print_(CN_n)

0.0, 0.3006128425639428, 1.0


In [41]:
from pyvis.network import Network
work_dir = '/home/nikolaisv/study/intership/visualization/'

In [1]:
class Graph:
    """
    Class for creating file congruence graph.
    Nodes representing users. Edge exists between nodes if users got common changed files. Edge color and width depends
    on number of common files between users.
    """

    def __init__(self, M, heading="Graph"):
        self.M = M
        self.user_to_id = load_json(f"{resources}/userToId")
        self.id_to_user = {v: k for k, v in self.user_to_id.items()}

        self.__init_net(heading)

    def __gen_quantiles(self):
        """
        Generate quantiles (0.25, 0.5, 0.75) for non null values
        """
        non_null_values = self.M[self.M != 0]
        self.quantiles = [
            np.quantile(non_null_values, 0.25)
            , np.quantile(non_null_values, 0.5)
            , np.quantile(non_null_values, 0.75)
        ]

    def __init_net(self, heading):
        """
        Create net-graph
        :param heading: heading
        """
        self.net = Network(heading=heading
                           , height="1080px"
                           , width="100%"
                           , bgcolor="#222222"
                           , font_color="white")

        self.__gen_quantiles()

        self.net.barnes_hut()

        size = self.M.shape[0]
        for i in range(size):
            for j in range(size):
                if i == j:
                    continue

                weight = int(self.M[i, j])
                if weight == 0:
                    continue
                src = self.id_to_user[i]
                dst = self.id_to_user[j]

                self.net.add_node(src, src, title=src)
                self.net.add_node(dst, dst, title=dst)
                self.net.add_edge(src, dst, value=weight, color=self.edge_color(weight, *self.quantiles))

        neighbor_map = self.net.get_adj_list()

        for node in self.net.nodes:
            node["title"] += " Neighbors:<br>" + "<br>".join(self.__pretty_string_neighbors(node, neighbor_map))
            node["value"] = len(neighbor_map[node["id"]])

    def __get_num_of_files_with_neighbor(self, user, neighbor):
        """
        Map function. Get number of common files with neighbor for user.
        :param user: user login
        :param neighbor: neighbor login
        :return: tuple (neighbor login, number of common files with neighbor)
        """
        num_of_files = self.M[self.user_to_id[user], self.user_to_id[neighbor]]
        return neighbor, num_of_files

    def __pretty_string_neighbors(self, node, neighbor_map):
        """
        Create string for better visualization.
        :param node: current node
        :param neighbor_map: neighbors for each node
        :return: string of neighbors and number of common files
        """
        result = map(lambda neighbor: self.__get_num_of_files_with_neighbor(node["title"], neighbor)
                     , neighbor_map[node["id"]])
        result = sorted(result, key=lambda x: x[1], reverse=True)
        result = map(lambda x: f"{x[0]} : {x[1]}", result)
        return result

    def show(self):
        self.net.show(f"{work_dir}/{self.net.heading}.html")

    @staticmethod
    def edge_color(weight, quantile1, quantile2, quantile3):
        if weight <= quantile1:
            return '#0569E1'
        if weight <= quantile2:
            return '#C1F823'
        if weight <= quantile3:
            return '#FCAA05'
        return '#EE5503'

In [43]:
graph = Graph(CN_n)
graph.show()